<a href="https://colab.research.google.com/github/deepsharma26/SIRT1_Main/blob/Descriptor_genration/RDkit_discriptors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 13.4 MB/s eta 0:00:00


In [2]:
!pip install mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176718 sha256=672520e06e501c41ab820c19df5f9be24ef5eb522ed274d34435bef3ffb18593
  Stored in directory: /root/.cache/pip/wheels/8b/30/0b/84e3f6775306e74cf5957ee4d16b10bf3927dcec44cc23d5f2
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux"

In [4]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [5]:
dataset=pd.read_csv('/content/SIRT1_04_bioactivity_data_3class_pIC50.csv')

In [6]:
df=dataset.dropna(subset=['canonical_smiles'])

In [7]:
df

,Unnamed: 0,molecule_chembl_id,canonical_smiles,Class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,0,CHEMBL420311,NC(=O)C1CCCc2c1[nH]c1ccc(Cl)cc21,Active,248.713,2.72650,2.0,1.0,10.008774
1,1,CHEMBL115600,Cc1ccc2[nH]c3c(c2c1)CCCC3C(N)=O,Active,228.295,2.38152,2.0,1.0,9.688246
2,2,CHEMBL112265,NC(=O)C1CCCc2c1[nH]c1ccccc21,Inactive,214.268,2.07310,2.0,1.0,8.832683
3,3,CHEMBL446446,NC(=O)C1CCCCc2c1[nH]c1ccc(Cl)cc21,Active,262.740,3.11660,2.0,1.0,9.906578
4,4,CHEMBL171137,CCOC(=O)C1CCCc2c1[nH]c1ccc(Cl)cc21,Inactive,277.751,3.80430,1.0,2.0,7.000000
...,...,...,...,...,...,...,...,...,...
928,928,CHEMBL4793948,O=C(Nc1ccccc1)Nc1ccc(NC(=O)c2ccccc2)cc1,Inactive,331.375,4.58290,3.0,2.0,8.920819
929,929,CHEMBL4749004,O=C(NCCc1c[nH]c2ccccc12)c1cc2ccccc2cc1O,Inactive,330.387,3.99920,3.0,2.0,8.602060
930,930,CHEMBL5416344,O=C(O)CCNC(=S)NCCCNc1nc(Nc2ccccc2Cl)ncc1C(=O)N...,Inactive,582.130,4.24410,5.0,7.0,6.948462
931,931,CHEMBL5424599,O=C(O)CCNC(=S)NCCCNc1nc(Nc2ccccc2)nc(Nc2ccccc2...,Inactive,501.016,3.75300,6.0,8.0,7.659556


In [8]:
def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors = []
    for mol in mols:
        # add hydrogens to molecules
        mol = Chem.AddHs(mol)
        # Calculate all 200 descriptors for each molecule
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors, desc_names

# Split the SMILES into chunks of 100,000 for faster processing
chunk_size = 100000
chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

total_chunks=len(chunks)
total_time=0

# Check if there is an existing output file
if os.path.isfile('RDkit_descriptors.csv'):
    existing_data = pd.read_csv('RDkit_descriptors.csv', index_col=0)
else:
    existing_data = pd.DataFrame()
    # Calculate descriptors for each chunk and concatenate the results
for i, chunk in enumerate(tqdm(chunks, desc='Processing', total=len(chunks))):
    # Check if this chunk has already been processed
    if len(existing_data) >= len(chunk):
        continue
    # Calculate descriptors for this chunk
    descriptors, desc_names = RDkit_descriptors(chunk['canonical_smiles'])
    # Convert the descriptors to a dataframe
    df_with_200_descriptors = pd.DataFrame(descriptors, columns=desc_names,)
    # Add the chunk index as a new column
    df_with_200_descriptors['chunk_index'] = i
    # Append the data to the existing data
    existing_data = pd.concat([existing_data, df_with_200_descriptors], axis=0)
    # Save the data after each chunk
    existing_data.to_csv('RDkit_descriptors.csv')

# Save the final data
existing_data.to_csv('RDkit_descriptors.csv')



Processing: 100%|██████████| 1/1 [00:50<00:00, 50.33s/it]
